In [ ]:
import mysql.connector
from nltk.tokenize import sent_tokenize
import import_ipynb
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from algoritma.MMR import *
from nltk.corpus import stopwords
import string
from termcolor import colored
import time

# Deklarasi Connection Database

Membuat koneksi kedatabase lokal dengan database 'db-berita-mca'

In [ ]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
) 

# Membuat Fungsi Steaming Use Sastrawi

Membuat stemmer pakai sastrawi 
Digunakan untuk melakukan steaming kalimat (menjadi kalimat dasar Menggunakan Library Sastrawi)

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Mengambil Data pada Database

Mengambil data pada database,data yang diambil hanya 2 kolom saja

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("SELECT id, content FROM tb_berita")
myresult = mycursor.fetchall()

df = pd.DataFrame(myresult,columns=['id','article'])
df

# Load Data Stopword Lib & File Internal

Mengambil data stopword bahasa indonesia yang pertama menggunakan file list dari stop word 917 dan menggunakan library nltk sebanyak panjang data 789

In [ ]:
def load_stopWords():
    r = open("stopwordlist-indonesia.txt", "r")
    readTxt = r.read()
    idStopWord = readTxt.split() 
    return idStopWord

In [ ]:
def load_stopWordsNLTK():
    stop_words = stopwords.words('indonesian')
    return stop_words

# Simpan hasil summary ke database

Menyimpan data yang telah dihasilakan melalui summary ke database dengan cara mengupdate data tersebut dan memasukan text summary pada field tersebut

In [ ]:
def simpanSummary(mydb,summary,id):
    mycursor = mydb.cursor()
    id = int(id)
    sql = "UPDATE tb_berita SET summary = %s WHERE id = %s"
    param = (summary,id)
    mycursor.execute(sql, param)
    mydb.commit()

# Proses Text Preprocessing 

[tokenizeParagraftoKalimat, cleansingData, steamingData, indoStopWord]

Fungsi diatas merupakan fungsi yang digunakan untuk melakukan preprocessing article sebuah berita, penjelasan mengenai fungsi akan dijelaskan lebih lanjut

[tokenizeParagraftoKalimat]

berfungsi sebagai tokenize pargraf menjadi bagian perkalimat 

In [ ]:
def tokenizeParagraftoKalimat(paragraf):
    tokenize = sent_tokenize(paragraf)
    return tokenize

[cleansingData]

berfungsi sebagai membersihkan data dari huruf tanda petik dan masih kata besar atau keceil

In [ ]:
def cleansingData(text):
    hasil = re.sub('[^A-Za-z0-9 ]+', '', text)
    hasil = hasil.translate(str.maketrans("","",string.punctuation))
    return hasil

[steamingData]

berguna sebagai menjadikan kata dasar sebuah kalimat dan fungsi stemmer tersebut sekaligus menjadikan kalimat menjadi lower atau huruf kecil

In [ ]:
def steamingData(text):
    steamingData = stemmer.stem(text)
    return steamingData

[indoStopWord]

berguna dalam menghilangkan kata sambung atau tidak diperlukan, ini berfungsi dlam menghitung kesamaan antar kalimat pada algoritma MMR agar lebih mudah dan mendapatkan hasil yang akurat (Lib NLTK & List Stopword)

In [ ]:
def indoStopWord(stopNLTK, stopTxt, kalimat):
    hasilStopWord = []
    for kata in kalimat.split():
        if not kata in stopNLTK:
            if not kata in stopTxt:
                hasilStopWord.append(kata)
    return " ".join(hasilStopWord)

[hasilSummary]

fungsi yang berguna dalam menampilkan hasil summarization pada algoritma MMR

In [ ]:
def hasilSummary(kalimatAsli, summarySet):
    hasil = []
    for sentence in summarySet:
        hasil.append(kalimatAsli [sentence].lstrip(' ')) 
    return " ".join(hasil)

[visualisasiSummaryOri]

menampilkan visualisasi yang sangat berguna dalam mengetahui kata mana saja yang dipakai atau ditampilkan pada summarization

In [ ]:
def visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli):
    for sentence in cleansingTeks:
        if sentence in summarySet:
            print(colored(kalimatAsli[sentence].lstrip(' '),'red'))
        else:
            print(kalimatAsli[sentence].lstrip(' '))

bisa dibilang sebagai code utama, dimana disini tempat menjalankan semua code dengan melakukan pemanggilan semua fungsi dan class algoritam MMR 

In [ ]:
for iteration, article in enumerate(df['article']):
    id = df['id'][iteration]
    tokenizeKalimat = tokenizeParagraftoKalimat(article)
    
    kalimat = []
    cleansingTeks = []
    kalimatAsli = {}

    for teks in tokenizeKalimat:
        kataDasar = steamingData(cleansingData(teks))
        dataFinal = indoStopWord(load_stopWordsNLTK(),load_stopWords(),kataDasar)
        cleansingTeks.append(dataFinal)
        kalimat.append(teks)
        kalimatAsli[dataFinal] = teks

    iteration = MMR()
    score = iteration.similarityScore(cleansingTeks)
    summarySet = iteration.calculateMMR(score,cleansingTeks)
    

    summary = hasilSummary(kalimatAsli,summarySet)
    simpanSummary(mydb,summary,id)
    print ('\nSummary dari Berita ID : {} (hasil teks yang diringkas):\n'.format(id))
    print(summary)

    print ('=============================================================')
    print ('\Oringinal Article (Teks Asli):\n')
    visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli)